# End of Epoch Simulations

Here are some simulations for how the end of the epoch will go. Here are some basic assumptions present here.
1. All active traders are a part of this competition, i.e. they are going to pay an optimal amount in fees for rewards.
2. Traders can be modeled as updating their expectation for the overall market summed trader score by using their old estimate of the total score and performing a learning update that incorporates a new estimate with their old estimate. In other words, we can model each participant as updating their expectations in parallel as the epoch draws to a close.
3. The overal market participants' distribution of open interest follows an exponential distribution. This is an approximation, and in fact exponential distributions don't have quite fat enough of a tail in order to work here.
4. Rational agents have access to infinite money. That is, they may adjust their fee-paying schedule to spend more than the amount they have in their collateral account. Practically, this is probably not the case. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from utils import *

In [ ]:
R = 3_835_616 
alpha = .01
open_interest = 150_000_000

In [ ]:
results = dict()
slope = dict()

for trial in range(5):
    d_mkt, f_mkt, g_mkt = find_equilibrium_stk(open_interest, n=1000, G=5000, R=R, p=20, alpha=alpha, num_whales=1, whale_alpha=10)
    results[trial] = sorted(list(zip(d_mkt, g_mkt, f_mkt)), key=lambda x : x[2]) 
    slope[trial] = sorted(list(zip(d_mkt**0.28 * g_mkt**0.05, f_mkt**0.33)), key=lambda x : x[1]) 

In [ ]:
for trial in range(5):
    plt.scatter(range(1000), [x[2]/x[0] for x in results[trial]])